# CIFAR-10 Classifier

## Import modules

In [1]:
print("hello")

hello


In [4]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import numpy as np
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


## Import data

In [30]:
# Code for unpacking data, taken from the CIFAR-10 homepage
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

batchNames = ['data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5']

# Training set
xTrainRaw = np.empty((0,3072),int)
yTrainRaw = np.empty((0))
# Loop through the files and create one large training set
for name in batchNames:
    
    trainBatch = unpickle("../data/" + name)
    xTrainRaw = np.append(xTrainRaw, trainBatch[b'data'], axis=0)
    yTrainRaw = np.append(yTrainRaw, trainBatch[b'labels'], axis=0)

test = unpickle('../data/test_batch')
print(xTrainRaw.shape)

# Test set
xTestRaw = test[b'data']
yTestRaw = np.asarray(test[b'labels'])

(50000, 3072)


## Data info

In [26]:
# Get data metadata
metadata = unpickle('../data/batches.meta')
print("Label names:", metadata[b'label_names'])


print("Data dimensions:", trainBatch1[b'data'].shape)

# Access the dict with a b' prefix
# Like this:
## batch_1[b'labels']
## batch_1[b'data'] 

Label names: [b'airplane', b'automobile', b'bird', b'cat', b'deer', b'dog', b'frog', b'horse', b'ship', b'truck']
Data dimensions: (10000, 3072)


Each batch dict contains four elements:
- **data**: The image data as Numpy arrays which are fed into a model (i.e. the x value). The data has dimensions 10,000x3,072, that is, 10,000 images with 32x32 pixels * 3 color channels (RGB)
- **labels**: The actual labels which the model must successfully classify (i.e. the y value)
- batch_label: The name of the batch (e.g. batch 1/2/3/) (not important)
- filenames: Name of each image (not important)

# Transform data

The x data in its raw form, (50000,3072) for the training set (10000,3072) for the test set, is not suitable for feeding into a model.

For a CNN, we should convert the dimensions of each image from a simple row vector (3072) into (width,height,nChannels), which in our case is (32,32,3). 

In [32]:
xTrain = xTrainRaw.reshape(50000, 32,32,3)
xTest = xTestRaw.reshape(10000,32,32,3)

yTrain = yTrainRaw
yTest = yTestRaw

# Build model

In [33]:
# Sequential CNN model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)               

# Train model

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(xTrain, yTrain, epochs=1000, 
                    validation_data=(xTest, yTest))

Epoch 1/1000
1563/1563 [==============================] - 4s 3ms/step - loss: 1.9191 - accuracy: 0.3333 - val_loss: 1.6787 - val_accuracy: 0.3919
Epoch 2/1000
1563/1563 [==============================] - 4s 3ms/step - loss: 1.5568 - accuracy: 0.4421 - val_loss: 1.5501 - val_accuracy: 0.4311
Epoch 3/1000
1563/1563 [==============================] - 4s 3ms/step - loss: 1.4607 - accuracy: 0.4778 - val_loss: 1.4389 - val_accuracy: 0.4882
Epoch 4/1000
1563/1563 [==============================] - 4s 3ms/step - loss: 1.4018 - accuracy: 0.5007 - val_loss: 1.3899 - val_accuracy: 0.5019
Epoch 5/1000
1563/1563 [==============================] - 4s 3ms/step - loss: 1.3555 - accuracy: 0.5206 - val_loss: 1.4142 - val_accuracy: 0.4917
Epoch 6/1000
1563/1563 [==============================] - 4s 3ms/step - loss: 1.3101 - accuracy: 0.5374 - val_loss: 1.3729 - val_accuracy: 0.5178
Epoch 7/1000
1563/1563 [==============================] - 4s 3ms/step - loss: 1.2794 - accuracy: 0.5478 - val_loss: 1.3770 -

# Evaluate model